In [1]:
# 
# 导入需要的包
import cv2
import os
import random
import paddle
from paddle.vision.transforms import ToTensor, Compose, Normalize, Resize, RandomRotation, RandomCrop, RandomHorizontalFlip, RandomErasing
from paddle.vision.transforms import ColorJitter, CenterCrop
import paddle.optimizer.lr as lrScheduler
from paddle.io import DataLoader, Dataset
import numpy as np
import paddle.nn as nn
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
import time
import zipfile

print("本项目基于Paddle的版本号为："+paddle.__version__)

本项目基于Paddle的版本号为：2.6.0


# 解压数据集并进行数据预处理

In [2]:
# # zip_src: 需要解压的文件路径
# # dst_dir: 解压后文件存放路径
# test_zip_src = 'data/data256722/cat_12_test.zip'
# train_zip_src = 'data/data256722/cat_12_train.zip'
# test_src = './data/'
# train_src = './data/'
# def unzip_file(zip_path, extract_to_path, encoding='utf-8'):
#     with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#         # 获取压缩包中的所有文件列表
#         file_list = zip_ref.namelist()

#         # 将文件列表按照指定编码解码为字符串
#         file_list_decoded = [file_name.encode(encoding).decode(encoding, 'replace') for file_name in file_list]

#         # 解压缩文件
#         for i, file_name in enumerate(file_list):
#             zip_ref.extract(file_name, extract_to_path)
#             # 重命名解压后的文件，使用解码后的文件名
#             extracted_path = os.path.join(extract_to_path, file_name)
#             renamed_path = os.path.join(extract_to_path, file_list_decoded[i])
#             os.rename(extracted_path, renamed_path)


# unzip_file(test_zip_src, test_src)
# unzip_file(train_zip_src, train_src)


## 数据预处理并进行数据集划分

In [3]:

#图像清洗
def img_resize(data_dir):
  print("文件路径下图像数量：", len(os.listdir(data_dir)))
  for img_name in os.listdir(data_dir):
    img_dir = os.path.join(data_dir, img_name)
    img = cv2.imread(img_dir)
    if img is None:
      continue
      #os.remove(img_dir)#清除脏数据
    else:
      img_resize = cv2.resize(img, (224, 224))#重塑图像尺寸
      cv2.imwrite(img_dir, img_resize)
img_resize("/home/aistudio/work/cat_12_train")


文件路径下图像数量： 2160


In [4]:

#数据集划分train：valid = 8 ：2
with open('/home/aistudio/work/train_list.txt', 'r') as f:
    image_list = f.readlines()
random.shuffle(image_list)
cut = int(len(image_list) * 0.8)
train_list = image_list[:cut]
valid_list = image_list[cut:]
      
#创建train_list.txt
with open('/home/aistudio/work/train.txt', 'w') as f:
    for path in train_list:
        img_paths = path.split()[0]
        img_label = path.split()[1]
        f.write(img_paths + ' ' + img_label + '\n')

#创建valid_list.txt
with open('/home/aistudio/work/valid.txt', 'w') as f:
    for path in valid_list:
        img_paths = path.split()[0]
        img_label = path.split()[1]
        f.write(img_paths + ' ' + img_label + '\n')


## 数据集构建

In [5]:
ResNetmean = [0.485, 0.456, 0.406]
ResNetstd = [0.229, 0.224, 0.225]
resnet_inv_mean = [-m/s for m, s in zip(ResNetmean, ResNetstd)]
resnet_inv_std = [1/s for s in ResNetstd]

ViTmean = [0.5, 0.5, 0.5]
ViTstd = [0.5, 0.5, 0.5]
vit_inv_mean = [-m/s for m, s in zip(ViTmean, ViTstd)]
vit_inv_std = [1/s for s in ViTstd]

BATCH_SIZE = 64
img_size = 224

data_dir = "/home/aistudio/work"
# image_to_label = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

ResNetTransformTrain = Compose([
    Resize((256, 256)), 
    CenterCrop((img_size, img_size)),
    ToTensor(), 
    Normalize(mean=ResNetmean,std=ResNetstd,data_format='CHW'),
    RandomErasing(prob=0.5, ratio=(0.01, 0.05))
])
ResNetTransformTest = Compose([
    Resize((img_size, img_size)),
    ToTensor(), 
    Normalize(mean=ResNetmean,std=ResNetstd,data_format='CHW')
])

ViTTransformTrain = Compose([
    Resize((256, 256)), 
    CenterCrop((img_size, img_size)),
    ToTensor(), 
    Normalize(mean=ViTmean,std=ViTstd,data_format='CHW'),
    RandomErasing(prob=0.5, ratio=(0.01, 0.05))
])
ViTTransformTest = Compose([
    Resize((img_size, img_size)),
    ToTensor(), 
    Normalize(mean=ViTmean,std=ViTstd,data_format='CHW')
])

inverse_ResNetTransform = Compose([Normalize(mean=resnet_inv_mean, std=resnet_inv_std, data_format='CHW')])
inverse_ViTTransform = Compose([Normalize(mean=vit_inv_mean, std=vit_inv_std, data_format='CHW')])

class CatDataset(Dataset):
    def __init__(self, data_dir, mode, transform):
        super(CatDataset, self).__init__()
        self.transform = transform
        self.data_dir = data_dir
        self.mode = mode
        self.images = []
        self.labels = []
        if mode == 'train':
            self.path = data_dir + '/train.txt'
        else:
            self.path = data_dir + '/valid.txt'
        with open(self.path, "r") as file:
            for line in file:
                image, label = line.split(' ')
                self.images.append(image)
                self.labels.append(int(label.split('\\')[0]))

    def __getitem__(self, index):
        # 返回数据与标签
        image = Image.open(self.data_dir + '/' + self.images[index]).convert('RGB')
        if image is None:
            print(self.data_dir + '/' + self.images[index])
            return None, None
        data = self.transform(image)
        label = paddle.to_tensor(self.labels[index])
        return data, label

    def __len__(self):
       return len(self.labels)

ResNetTrainDataset = CatDataset(data_dir=data_dir, mode='train', transform=ResNetTransformTrain)
ResNetTestDataset = CatDataset(data_dir=data_dir, mode='test', transform=ResNetTransformTest)

ViTTrainDataset = CatDataset(data_dir=data_dir, mode='train', transform=ViTTransformTrain)
ViTTestDataset = CatDataset(data_dir=data_dir, mode='test', transform=ViTTransformTest)

ResNetTrainDataLoader = DataLoader(ResNetTrainDataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, drop_last=False)
ResNetTestDataLoader = DataLoader(ResNetTestDataset, batch_size=BATCH_SIZE*2, shuffle=False, num_workers=2, drop_last=False)

ViTTrainDataLoader = DataLoader(ViTTrainDataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, drop_last=False)
ViTTestDataLoader = DataLoader(ViTTestDataset, batch_size=BATCH_SIZE*2, shuffle=False, num_workers=2, drop_last=False)

# 定义网络

## ResNet

In [15]:
resnetmodel = paddle.vision.models.resnet50(pretrained=True, num_classes=12)
for param in resnetmodel.parameters():
    param.stop_gradient = False

100%|██████████| 151272/151272 [00:01<00:00, 99970.95it/s] 
/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/paddle/nn/layer/layers.py:2084: UserWarning: Skip loading for fc.weight. fc.weight receives a shape [2048, 1000], but the expected shape is [2048, 12].
  warnings.warn(f"Skip loading for {key}. " + str(err))
/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/paddle/nn/layer/layers.py:2084: UserWarning: Skip loading for fc.bias. fc.bias receives a shape [1000], but the expected shape is [12].
  warnings.warn(f"Skip loading for {key}. " + str(err))


## ViT

In [7]:
# 图像分块、Embedding
class PatchEmbed(nn.Layer):
    def __init__(self, img_size=224, patch_size=16, in_chans=3, embed_dim=768):
        super().__init__()
        # 原始大小为int，转为tuple，即：img_size原始输入224，变换后为[224,224]
        img_size = to_2tuple(img_size)
        patch_size = to_2tuple(patch_size)
        # 图像块的个数
        num_patches = (img_size[1] // patch_size[1]) * \
            (img_size[0] // patch_size[0])
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = num_patches
        # kernel_size=块大小，即每个块输出一个值，类似每个块展平后使用相同的全连接层进行处理
        # 输入维度为3，输出维度为块向量长度
        # 与原文中：分块、展平、全连接降维保持一致
        # 输出为[B, C, H, W]
        self.proj = nn.Conv2D(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        B, C, H, W = x.shape
        assert H == self.img_size[0] and W == self.img_size[1], \
            "Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]})."
        # [B, C, H, W] -> [B, C, H*W] ->[B, H*W, C]
        x = self.proj(x).flatten(2).transpose((0, 2, 1))
        return x


# Multi-head Attention
class Attention(nn.Layer):
    def __init__(self,
                 dim,
                 num_heads=8,
                 qkv_bias=False,
                 qk_scale=None,
                 attn_drop=0.,
                 proj_drop=0.):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim**-0.5
        # 计算 q,k,v 的转移矩阵
        self.qkv = nn.Linear(dim, dim * 3, bias_attr=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        # 最终的线性层
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, x):
        N, C = x.shape[1:]
        # 线性变换
        qkv = self.qkv(x).reshape((-1, N, 3, self.num_heads, C //
                                   self.num_heads)).transpose((2, 0, 3, 1, 4))
        # 分割 query key value
        q, k, v = qkv[0], qkv[1], qkv[2]
        # Scaled Dot-Product Attention
        # Matmul + Scale
        attn = (q.matmul(k.transpose((0, 1, 3, 2)))) * self.scale
        # SoftMax
        attn = nn.functional.softmax(attn, axis=-1)
        attn = self.attn_drop(attn)
        # Matmul
        x = (attn.matmul(v)).transpose((0, 2, 1, 3)).reshape((-1, N, C))
        # 线性变换
        x = self.proj(x)
        x = self.proj_drop(x)
        return x


class Mlp(nn.Layer):
    def __init__(self,
                 in_features,
                 hidden_features=None,
                 out_features=None,
                 act_layer=nn.GELU,
                 drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        # 输入层：线性变换
        x = self.fc1(x)
        # 应用激活函数
        x = self.act(x)
        # Dropout
        x = self.drop(x)
        # 输出层：线性变换
        x = self.fc2(x)
        # Dropout
        x = self.drop(x)
        return x


def drop_path(x, drop_prob=0., training=False):
    if drop_prob == 0. or not training:
        return x
    keep_prob = paddle.to_tensor(1 - drop_prob)
    shape = (paddle.shape(x)[0], ) + (1, ) * (x.ndim - 1)
    random_tensor = keep_prob + paddle.rand(shape, dtype=x.dtype)
    random_tensor = paddle.floor(random_tensor)
    output = x.divide(keep_prob) * random_tensor
    return output

class DropPath(nn.Layer):
    def __init__(self, drop_prob=None):
        super(DropPath, self).__init__()
        self.drop_prob = drop_prob

    def forward(self, x):
        return drop_path(x, self.drop_prob, self.training)


class Block(nn.Layer):
    def __init__(self,
                 dim,
                 num_heads,
                 mlp_ratio=4.,
                 qkv_bias=False,
                 qk_scale=None,
                 drop=0.,
                 attn_drop=0.,
                 drop_path=0.,
                 act_layer=nn.GELU,
                 norm_layer='nn.LayerNorm',
                 epsilon=1e-5):
        super().__init__()
        self.norm1 = eval(norm_layer)(dim, epsilon=epsilon)
        # Multi-head Self-attention
        self.attn = Attention(
            dim,
            num_heads=num_heads,
            qkv_bias=qkv_bias,
            qk_scale=qk_scale,
            attn_drop=attn_drop,
            proj_drop=drop)
        # DropPath
        self.drop_path = DropPath(drop_path) if drop_path > 0. else Identity()
        self.norm2 = eval(norm_layer)(dim, epsilon=epsilon)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim,
                       hidden_features=mlp_hidden_dim,
                       act_layer=act_layer,
                       drop=drop)

    def forward(self, x):
        # Multi-head Self-attention， Add， LayerNorm
        x = x + self.drop_path(self.attn(self.norm1(x)))
        # Feed Forward， Add， LayerNorm
        x = x + self.drop_path(self.mlp(self.norm2(x)))
        return x


# 参数初始化配置
trunc_normal_ = nn.initializer.TruncatedNormal(std=.02)
zeros_ = nn.initializer.Constant(value=0.)
ones_ = nn.initializer.Constant(value=1.)

# 将输入 x 由 int 类型转为 tuple 类型
def to_2tuple(x):
    return tuple([x] * 2)

# 定义一个什么操作都不进行的网络层
class Identity(nn.Layer):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, input):
        return input


class VisionTransformer(nn.Layer):
    def __init__(self,
                 img_size=224,
                 patch_size=16,
                 in_chans=3,
                 class_dim=1000,
                 embed_dim=768,
                 depth=12,
                 num_heads=12,
                 mlp_ratio=4,
                 qkv_bias=False,
                 qk_scale=None,
                 drop_rate=0.,
                 attn_drop_rate=0.,
                 drop_path_rate=0.,
                 norm_layer='nn.LayerNorm',
                 epsilon=1e-5,
                 **args):
        super().__init__()
        self.class_dim = class_dim

        self.num_features = self.embed_dim = embed_dim
        # 图片分块和降维，块大小为patch_size，最终块向量维度为768
        self.patch_embed = PatchEmbed(
            img_size=img_size,
            patch_size=patch_size,
            in_chans=in_chans,
            embed_dim=embed_dim)
        # 分块数量
        num_patches = self.patch_embed.num_patches
        # 可学习的位置编码
        self.pos_embed = self.create_parameter(
            shape=(1, num_patches + 1, embed_dim), default_initializer=zeros_)
        self.add_parameter("pos_embed", self.pos_embed)
        # 人为追加class token，并使用该向量进行分类预测
        self.cls_token = self.create_parameter(
            shape=(1, 1, embed_dim), default_initializer=zeros_)
        self.add_parameter("cls_token", self.cls_token)
        self.pos_drop = nn.Dropout(p=drop_rate)

        dpr = np.linspace(0, drop_path_rate, depth)
        # transformer
        self.blocks = nn.LayerList([
            Block(
                dim=embed_dim,
                num_heads=num_heads,
                mlp_ratio=mlp_ratio,
                qkv_bias=qkv_bias,
                qk_scale=qk_scale,
                drop=drop_rate,
                attn_drop=attn_drop_rate,
                drop_path=dpr[i],
                norm_layer=norm_layer,
                epsilon=epsilon) for i in range(depth)
        ])

        self.norm = eval(norm_layer)(embed_dim, epsilon=epsilon)

        # Classifier head
        self.head = nn.Linear(embed_dim,
                              class_dim) if class_dim > 0 else Identity()

        trunc_normal_(self.pos_embed)
        trunc_normal_(self.cls_token)
        self.apply(self._init_weights)

    # 参数初始化
    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight)
            if isinstance(m, nn.Linear) and m.bias is not None:
                zeros_(m.bias)
        elif isinstance(m, nn.LayerNorm):
            zeros_(m.bias)
            ones_(m.weight)

    def forward_features(self, x):
        B = paddle.shape(x)[0]
        # 将图片分块，并调整每个块向量的维度
        x = self.patch_embed(x)
        # 将class token与前面的分块进行拼接
        cls_tokens = self.cls_token.expand((B, -1, -1))
        x = paddle.concat((cls_tokens, x), axis=1)
        # 将编码向量中加入位置编码
        x = x + self.pos_embed
        x = self.pos_drop(x)
        # 堆叠 transformer 结构
        for blk in self.blocks:
            x = blk(x)
        # LayerNorm
        x = self.norm(x)
        # 提取分类 tokens 的输出
        return x[:, 0]

    def forward(self, x):
        # 获取图像特征
        x = self.forward_features(x)
        # 图像分类
        x = self.head(x)
        return x

vitmodel = VisionTransformer(
    patch_size=16,
    embed_dim=768,
    depth=8,
    class_dim=12,
    num_heads=8,
    mlp_ratio=3,
    qk_scale=768 ** -0.5,
    )
param = paddle.load('/home/aistudio/work/ViT_small_patch16_224_pretrained.pdparams')
vitmodel.set_state_dict(param)
paddle.summary(vitmodel, (4, 3, 224, 224))

W0113 14:23:03.896580   278 gpu_resources.cc:119] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 12.0, Runtime API Version: 11.8
W0113 14:23:03.897814   278 gpu_resources.cc:164] device: 0, cuDNN Version: 8.9.
/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/paddle/nn/layer/layers.py:2084: UserWarning: Skip loading for head.weight. head.weight receives a shape [768, 1000], but the expected shape is [768, 12].
  warnings.warn(f"Skip loading for {key}. " + str(err))
/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/paddle/nn/layer/layers.py:2084: UserWarning: Skip loading for head.bias. head.bias receives a shape [1000], but the expected shape is [12].
  warnings.warn(f"Skip loading for {key}. " + str(err))


---------------------------------------------------------------------------
 Layer (type)       Input Shape          Output Shape         Param #    
   Conv2D-1      [[4, 3, 224, 224]]    [4, 768, 14, 14]       590,592    
 PatchEmbed-1    [[4, 3, 224, 224]]     [4, 196, 768]            0       
   Dropout-1      [[4, 197, 768]]       [4, 197, 768]            0       
  LayerNorm-1     [[4, 197, 768]]       [4, 197, 768]          1,536     
   Linear-1       [[4, 197, 768]]       [4, 197, 2304]       1,769,472   
   Dropout-2     [[4, 8, 197, 197]]    [4, 8, 197, 197]          0       
   Linear-2       [[4, 197, 768]]       [4, 197, 768]         590,592    
   Dropout-3      [[4, 197, 768]]       [4, 197, 768]            0       
  Attention-1     [[4, 197, 768]]       [4, 197, 768]            0       
  Identity-1      [[4, 197, 768]]       [4, 197, 768]            0       
  LayerNorm-2     [[4, 197, 768]]       [4, 197, 768]          1,536     
   Linear-3       [[4, 197, 768]]   

{'total_params': 47842572, 'trainable_params': 47842572}

# 训练

In [17]:
# 带Warmup的Cosine学习率衰减方式
def get_scheduler(epochs, warmup_epochs, learning_rate):
    base_scheduler = lrScheduler.CosineAnnealingDecay(learning_rate=learning_rate, T_max=epochs, eta_min=1e-5, verbose=False)
    scheduler = lrScheduler.LinearWarmup(base_scheduler, warmup_epochs, 1e-5, learning_rate, last_epoch=-1, verbose=False)
    return scheduler

def train_epoch(model, train_loader, epoch):
    model.train()
    acc_num = 0
    total_samples = 0
    nb = len(train_loader)
    pbar = enumerate(train_loader)
    pbar = tqdm(pbar, total=nb, colour='red')
    pbar.set_description(f'EPOCH: {epoch:3d}')
    for _, (_, data) in enumerate(pbar):
        x_data = data[0]
        y_data = data[1]
        predicts = model(x_data)
        # print(predicts.shape, y_data.shape)
        loss = criterion(predicts, y_data)
        loss_item = loss.item()
        acc_num += paddle.sum(predicts.argmax(1) == y_data.squeeze()).item()
        total_samples += y_data.shape[0]
        total_acc = acc_num / total_samples
        current_lr = optim.get_lr()
        loss.backward()
        pbar.set_postfix(train_loss=f'{loss_item:5f}', train_acc=f'{total_acc:5f}', train_lr=f'{current_lr:5f}')
        optim.step()
        optim.clear_grad()
    scheduler.step()

@paddle.no_grad()
def validation(model, test_loader):
    model.eval()
    acc_num = 0
    total_samples = 0
    nb = len(test_loader)
    pbar = enumerate(test_loader)
    pbar = tqdm(pbar, total=nb, colour='green')
    pbar.set_description(f'EVAL')
    for _, (_, data) in enumerate(pbar):
        x_data = data[0]
        y_data = data[1]
        predicts = model(x_data)
        acc_num += paddle.sum(predicts.argmax(1) == y_data.squeeze()).item()
        total_samples += y_data.shape[0]
        y_data = paddle.unsqueeze(y_data, axis=1)
        batch_acc = paddle.metric.accuracy(predicts, y_data).item()
        total_acc = acc_num / total_samples
        pbar.set_postfix(eval_batch_acc=f'{batch_acc:4f}', total_acc=f'{total_acc:4f}')
    return total_acc

def train(Model, TrainDataLoader, TestDataLoader, save_dir):
    # Model.set_state_dict(paddle.load('{}/best.pdparams'.format(save_dir)))
    best_acc = 0.8#validation(Model, TestDataLoader)
    # Model.set_state_dict(paddle.load('{}/best.pdparams'.format(save_dir)))
    start = time.time()
    for epoch in range(EPOCHS):
        train_epoch(Model, TrainDataLoader, epoch)
        acc = validation(Model, TestDataLoader)
        if acc > best_acc:
            paddle.save(Model.state_dict(), '{}/best.pdparams'.format(save_dir))
            best_acc = acc
        if (epoch + 1) % 10 == 0:
            paddle.save(Model.state_dict(),'{}/epoch_'.format(save_dir)+ str(epoch + 1) + '.pdparams')
    paddle.save(Model.state_dict(), '{}/finished.pdparams'.format(save_dir))
    end = time.time()
    print('Training Cost ', (end-start) / 60, 'minutes')

# 加载预训练模型

In [9]:
# def ViT_small_patch16_224(pretrained=False, use_ssld=False, **kwargs):
#     model = VisionTransformer(
#         patch_size=16,
#         embed_dim=768,
#         depth=8,
#         num_heads=8,
#         mlp_ratio=3,
#         qk_scale=768**-0.5,
#         **kwargs)
#     _load_pretrained(
#         pretrained,
#         model,
#         MODEL_URLS["ViT_small_patch16_224"],
#         use_ssld=use_ssld)
#     return model

In [18]:
# hyperparameter
EPOCHS = 5
NUM_CLASSES = 12
WARMUP_EPOCHS = 10
LR = 0.00005
SGD_LR = 0.001

# 手动切换哪个具体模型Model = resnetmodel
Model = vitmodel
scheduler = get_scheduler(epochs=EPOCHS, warmup_epochs=WARMUP_EPOCHS, learning_rate=LR)
# ResNet
#optim = paddle.optimizer.Adam(learning_rate=scheduler, parameters=Model.parameters(), weight_decay=5e-4)
# ViT
optim = paddle.optimizer.SGD(learning_rate=SGD_LR, parameters=Model.parameters())
criterion = paddle.nn.CrossEntropyLoss()

In [19]:
# ResNet
train(Model, ResNetTrainDataLoader, ResNetTestDataLoader, './work/ResNet')

EVAL: 100%|██████████| 4/4 [00:02<00:00,  1.54it/s, eval_batch_acc=0.958333, total_acc=0.962963]


Training Cost  1.1328781644503276 minutes


In [21]:
# ViT
train(Model, ViTTrainDataLoader, ViTTestDataLoader, './work/ViT')

EVAL: 100%|██████████| 4/4 [00:02<00:00,  1.50it/s, eval_batch_acc=0.958333, total_acc=0.969907]


Training Cost  1.1787092804908752 minutes


In [20]:
state_dict = paddle.load('./work/ResNet/best.pdparams')
Model.set_state_dict(state_dict)
Model.eval()
validation(Model, ResNetTestDataLoader)

EVAL: 100%|██████████| 4/4 [00:02<00:00,  1.51it/s, eval_batch_acc=0.958333, total_acc=0.965278]


0.9652777777777778

In [22]:
state_dict = paddle.load('./work/ViT/best.pdparams')
Model.set_state_dict(state_dict)
Model.eval()
validation(Model, ViTTestDataLoader)

EVAL: 100%|██████████| 4/4 [00:02<00:00,  1.52it/s, eval_batch_acc=0.958333, total_acc=0.969907]


0.9699074074074074